# Quantum Circuits for Calculating Molecules PES


We want to use a Quantum computer to compute the PES of a molecule.
To do so we must first obtain the molecules Hamiltonian which describes its energy and transform it such that we can encode a quantum computer circuit. 

We have two cooperating techniques in performing the transformations:

    Qubit Coupled Cluster (QCC)
    Qubit-wise commuting (QWC)

Once these transformations applied we build a quantum circuit and submit it to a quantum device for processing.



# First Section is the original task 5 - See below for Work Zone


# Quantum Circuits
Quantum computers can only use a specific set of gates (universal gate set). Given the entanglers and their amplitudes found in Step 3, one can find corresponding representation of these operators in terms of elementary gates using the following procedure.

In [1]:
import tequila as tq
from utility import *
from tqdm.auto import tqdm # pip install tqdm

First, we set up the Hamiltonian in Tequila's format and the unitary gates obtained in Step 3. 

In [2]:
H = tq.QubitHamiltonian.from_openfermion(get_qubit_hamiltonian('h2', 2, 'sto-3g', qubit_transf='jw')) 

# Should taper here from Step 4

a = tq.Variable("tau_0")
U = construct_QMF_ansatz(4) # Why 4?
U += tq.gates.ExpPauli(paulistring=tq.PauliString.from_string("X(0)Y(1)X(2)X(3)"), angle=a)
print(U)

circuit: 
Rx(target=(0,), parameter=beta_0)
Rz(target=(0,), parameter=gamma_0)
Rx(target=(1,), parameter=beta_1)
Rz(target=(1,), parameter=gamma_1)
Rx(target=(2,), parameter=beta_2)
Rz(target=(2,), parameter=gamma_2)
Rx(target=(3,), parameter=beta_3)
Rz(target=(3,), parameter=gamma_3)
Exp-Pauli(target=(0, 1, 2, 3), control=(), parameter=tau_0, paulistring=X(0)Y(1)X(2)X(3))



One can check the expectation value to see it is near the ground state energy.

In [3]:
E = tq.ExpectationValue(H=H, U=U)
vars = {'beta_1': 3.141592624143881, 'beta_0': 3.141592624143881, 'tau_0': 1.1331410014096885, 'gamma_1': 0.0, 'beta_3': 0.0, 'gamma_3': 0.0, 'gamma_2': 0.0, 'gamma_0': 0.0, 'beta_2': 0.0} # values obtained from step 3
print(tq.simulate(E, variables=vars))

-0.9486411121761622


One can run the same experiment on a real quantum computer through IBM Quantum Experience (ibmq). After activating your account here (https://quantum-computing.ibm.com/login), copy the API token and execute the commented block below. 

In [4]:
from qiskit import IBMQ
IBMQ.save_account('')

IBMQAccountCredentialsInvalidToken: 'Invalid IBM Quantum Experience token found: "" of type <class \'str\'>.'

In [ ]:
# list of devices available can be found in ibmq account page
tq.simulate(E, variables=vars, samples=100, backend="qiskit", device='ibmq_qasm_simulator')

The following code block prints the circuit.

In [4]:
circ = tq.circuit.compiler.compile_exponential_pauli_gate(U)
tq.draw(circ, backend="qiskit")

     ┌────────────────────┐┌─────────────────────┐   ┌───┐                   »
q_0: ┤ RX(f((beta_0,))_0) ├┤ RZ(f((gamma_0,))_1) ├───┤ H ├──────■────────────»
     ├────────────────────┤├─────────────────────┤┌──┴───┴───┐┌─┴─┐          »
q_1: ┤ RX(f((beta_1,))_2) ├┤ RZ(f((gamma_1,))_3) ├┤ RX(pi/2) ├┤ X ├──■───────»
     ├────────────────────┤├─────────────────────┤└──┬───┬───┘└───┘┌─┴─┐     »
q_2: ┤ RX(f((beta_2,))_4) ├┤ RZ(f((gamma_2,))_5) ├───┤ H ├─────────┤ X ├──■──»
     ├────────────────────┤├─────────────────────┤   ├───┤         └───┘┌─┴─┐»
q_3: ┤ RX(f((beta_3,))_6) ├┤ RZ(f((gamma_3,))_7) ├───┤ H ├──────────────┤ X ├»
     └────────────────────┘└─────────────────────┘   └───┘              └───┘»
c_0: ════════════════════════════════════════════════════════════════════════»
                                                                             »
c_1: ════════════════════════════════════════════════════════════════════════»
                                                    

# Work zone - Project 2 - Task 5


In [2]:
class MeasurementGrouper():
    """
    Usage:
        # run the grouping algorithm
        h2o_mcalc = MeasurementGrouper('h2o')
        # get commuting groups
        comm_groups = h2o_mcalc.comm_groups
        # get qwc unitaries
        uqwcs = h2o_mcalc.uqwcs
        # get qwc groups
        qwc_groups = h2o_mcalc.qwc_groups
        # get z unitaries
        uqwcs = h2o_mcalc.uzs
        # get z groups
        z_groups = h2o_mcalc.z_groups
    """
    
    def __init__(self, mol_sym, basis='sto3g', mapping='jw', do_taper=True, apply_uzs=False, verbose=True):
        """
        parameters:
            mol_sym: string representing the molecule type

            basis: orbital basis
            mapping: fermion -> qubit mapping. 'jw' (jordan-wigner) is defualt
            do_taper: whether to do tapering after the mapping
            simulate_gates: whether to apply last set of unitaries for transforming
                qwc groups to z-basis. This is not actually needed if all you want
                is the unitaries, and is only for informational purposes. It takes
                a while to run on n2 and nh3
            verbose: whether to print steps
        """
        
        self.verbose = verbose
        
        # just a quick check to make sure the molecule is implemented
        valid_mol_syms = ['h2', 'h2o', 'lih', 'n2', 'nh3', 'h4']
        assert mol_sym in valid_mol_syms, \
            f"Provided invalid mol_sym: '{mol_sym}'. Please provide a mol_sym in {valid_mol_syms}"
        
        # lookup tables
        # choosing the minima from task 1 but not sure this is the right thing to do
        self.lut_geos = {
            'h2': 0.7,
            'h2o': 1,
            'lih': 1.5,
            'n2': 1.2,
            'nh3': 1.1,
            'h4': 95 # not the minimum, chose an angle
        }
        # literally counting the number of electrons from periodic table
        # but not sure this is the right way to do it
        self.lut_n_electrons = {
            'h2': 2,
            'h2o': 10,
            'lih': 4,
            'n2': 14,
            'nh3': 10,
            'h4': 4
        }
    
        # NOW FOLOW THE STEPS
        self._print("1) Initialise the molecule object in a given basis and with a provided mapping")
        self.mol = get_qubit_hamiltonian(
            mol=mol_sym, geometry=self.lut_geos[mol_sym], basis=basis, qubit_transf=mapping)
        n_orbitals = self._get_num_orbitals(self.mol)
        self._print(f"\t{n_orbitals} orbitals")
        
        self._print("2) Optionally apply tapering")
        if do_taper:
            self.mol = taper_hamiltonian(self.mol, n_spin_orbitals=self._get_num_orbitals(self.mol),
                                        n_electrons=self.lut_n_electrons[mol_sym], qubit_transf=mapping)
            n_orbitals = self._get_num_orbitals(self.mol)
            self._print(f"\t{n_orbitals} orbitals")
        
        if n_orbitals > 8:
            self._print("More than 8 orbitals: skipping steps 3), 4) and 5) and directly getting qwc groups")
            qwc_groups = get_qwc_group(self.mol)
            # switch structure for consistency with comm_groups structure
            self.qwc_groups = {i+1: v for i, v in enumerate(qwc_groups)}
            self._print(f"\t{len(self.qwc_groups)} QWC groups," \
                + f" a {len(self.mol.terms)/ len(self.qwc_groups):.2f} x reduction from {len(self.mol.terms)} terms")
        else:
            self._print("3) Get commuting groups")
            self.comm_groups = get_commuting_group(self.mol)
            self._print(
                f"\t{len(self.comm_groups)} commuting groups," \
                + f" a {len(self.mol.terms)/ len(self.comm_groups):.2f} x reduction from {len(self.mol.terms)} terms")

            self._print("4) For each commuting group get the unitary for transforming to qwc")
            self.uqwcs = {}
            for group_ix in tqdm(self.comm_groups, disable=(not self.verbose)): # comm groups is a dict with int keys
                self.uqwcs[group_ix] = get_qwc_unitary(self.comm_groups[group_ix])
        
            self._print("5) Apply unitaries to get the qwcs")
            self.qwc_groups = {}
            for group_ix in tqdm(self.comm_groups, disable=(not self.verbose)):
                self.qwc_groups[group_ix] = remove_complex(
                    self.uqwcs[group_ix] * self.comm_groups[group_ix] * self.uqwcs[group_ix])
            
        self._print("6) Get unitaries for rotating everything to the measurable z-basis")
        self.uzs = {}
        for group_ix in tqdm(self.qwc_groups, disable=(not self.verbose)):
            self.uzs[group_ix] = get_zform_unitary(self.qwc_groups[group_ix])
        
        self.z_groups = {}
        if apply_uzs:
            self._print("7) Finally, apply latter unitaries to move all qwc groups to z-basis")
            for group_ix in tqdm(self.qwc_groups, disable=(not self.verbose)):
                self.z_groups[group_ix] = remove_complex(
                    self.uzs[group_ix] * self.qwc_groups[group_ix] * self.uzs[group_ix])
        
    def _print(self, string):
        if self.verbose:
            print(string)
    
    def _get_num_orbitals(self, mol):
        """
        Hacky way to get number of orbitals for a molecule object
        Probably there is a better way to do it
        """
        max_orbital = 0
        for product_op in mol.terms:
            if product_op:
                mx = max([op[0] for op in product_op])
                if mx > max_orbital:
                    max_orbital = mx
        return max_orbital + 1

In [15]:
import tequila as tq
from utility import *

#
# Molecules to process
#

class p2t5:
    def __init__(self):
        self.molecules = [] 
        self.molecules.append("h2".upper())
        self.molecules.append("h2o".upper())
        self.molecules.append("lih".upper())
        self.molecules.append("h4".upper())
        self.molecules.append("n2".upper())
        self.molecules.append("nh3".upper())
        print(self.molecules)
        
    def haveMol(self,name):

        name = name.upper()
        for m in range(len(self.molecules)):
            if ( self.molecules[m] == name ):
                return True
        return False

    # Experienting zone to abstract parameters for various molecules 
    # We prepare entanglers using  QCC and QWC (from Step 3 and Step 4)

    def prepareEntanglers(self,molname,geometry=2.5,basis='sto-3g',n_ents=1,method="BFGS", qubit_transf = 'jw', active_orbitals=None, threshold=1e-6):
            if ( self.haveMol(molname)):

                print("Mapping selected: {}".format(qubit_transf))
                # get the measurements - Allows us to access geometries
                
                self.mcalc = MeasurementGrouper(molname,mapping = qubit_transf)
                geometry = self.mcalc.lut_geos[molname]
                print("Obtained geometry for {} is {}".format(molname, geometry))

                print("\n\nCalculating QCC for ", molname.upper())
                print("Basis: {}\nGeometry: {}\nEntanglers: {}\nActive Orbitals: {}".format(basis,geometry,n_ents,active_orbitals))
                
                # 
                
                xyz_data = get_molecular_data(molname, geometry=geometry, xyz_format=True)
                mol = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set=basis, active_orbitals = active_orbitals)
                hf_reference = hf_occ(2*mol.n_orbitals, mol.n_electrons)
                
                # FCI energy calculation
                
                E_FCI = mol.compute_energy(method='fci')
                
                # Hamiltonian
                
                H = mol.make_hamiltonian()

                print("\nHamiltonian has {} terms\n".format(len(H)))

                #Define number of entanglers to enter ansatz
                #Rank entanglers using energy gradient criterion
                ranked_entangler_groupings = generate_QCC_gradient_groupings(H.to_openfermion(), 
                                                                             2*mol.n_orbitals, 
                                                                             hf_reference, 
                                                                             cutoff=threshold)

                #print('Grouping gradient magnitudes (Grouping : Gradient magnitude):')
                #for i in range(len(ranked_entangler_groupings)):
                #    print('{} : {}'.format(i+1,ranked_entangler_groupings[i][1]))


                entanglers = get_QCC_entanglers(ranked_entangler_groupings, n_ents, 2*mol.n_orbitals)

                #print('\nSelected entanglers:')
                #for ent in entanglers:
                #    print(ent)

                #Mean-field part of U (Omega):    
                U_MF = construct_QMF_ansatz(n_qubits = 2*mol.n_orbitals)
                #Entangling part of U:
                U_ENT = construct_QCC_ansatz(entanglers)
                U_QCC = U_MF + U_ENT                    

                E = tq.ExpectationValue(H=H, U=U_QCC)
                
                print("Full QCC Circuit from Step 3: \n", U_QCC)

                initial_vals = init_qcc_params(hf_reference, E.extract_variables())
                print("Initial Values: {}".format(initial_vals))

                #Minimize wrt the entangler amplitude and MF angles:
                result = tq.minimize(objective=E, method=method, initial_values=initial_vals, tol=1.e-6)

                print('\nObtained QCC energy for {} ({} entanglers): {}'.format(molname.upper(),len(entanglers), result.energy))
                print('Compare to FCI energy: {}\n'.format(E_FCI))
                
                # TODO: ================= Construct entanglers from mcalc (task 4) ========================================
                
                #print('First commuting group')
                #print(mcalc.comm_groups[1])
                #print('\n')
                #print('First QWC group')
                #print(mcalc.qwc_groups[1])
                #print('\n')
                #print('First Z group')
                #print(mcalc.z_groups[1])
                #print('\n')
                
                # Possibly on ly need these
                #print('First qwc unitary')
                #print(mcalc.uqwcs[1])
                #print('\n')
                #print('First z unitary')
                #print(mcalc.uzs[1])

                
                U_QWC = None
                #if hasattr(self.mcalc, 'uqwcs'):
                #    U_QWC += self.mcalc.uqwcs[1]
                #if hasattr(self.mcalc, 'uzs'):
                #    U_QWC += self.mcalc.uzs[1]

                #print(U_QWC)

                
                # 
                # Perform the similar operation as U_MF + U_ENT using the mcalc entanglers
                #
                # We then need to make a choice: Do we compare both methods, two circuits? Or pick one method?
                
                mcalc_entanglers = []
                mcalc_entanglers.append(self.mcalc.uqwcs[1])
                mcalc_entanglers.append(self.mcalc.uzs[1])
                print("\nMCalc Entanglers: \n", mcalc_entanglers)
                
                # Memorize the last compute
                
                self.comp_n_ents = n_ents
                self.comp_basis = basis
                self.comp_mol = molname
                self.comp_E_QCC = result.energy
                self.comp_U_QCC = U_QCC                   # Entanglers from team step 3.
                #self.comp_entanglers = entanglers        # Entanglers from my version of Step 3. Replace with U_QCC
                self.comp_U_QWC = None                    # Build the entanglers from mcalc
                self.comp_E_FCI = E_FCI
                self.comp_initial_vals = initial_vals
                self.comp_qubit_transf = qubit_transf
                
            else:
                print("molecule not known")

    def calculateCircuit(self ):
        # Utilize results from calculate_QCC_energy
        
        print("Constructing circuit(s) for {}".format(self.comp_mol.upper()))
        
        H = tq.QubitHamiltonian.from_openfermion(get_qubit_hamiltonian(self.comp_mol, 2, self.comp_basis, qubit_transf=self.comp_qubit_transf)) 
        
        print(self.mcalc)
        #print('\n')
        #print('First commuting group')
        #print(mcalc.comm_groups[1])
        #print('\n')
        #print('First QWC group')
        #print(mcalc.qwc_groups[1])
        #print('\n')
        #print('First Z group')
        #print(mcalc.z_groups[1])
        #print('\n')

        #print('First qwc unitary')
        #print(mcalc.uqwcs[1])
        #print('\n')
        #print('First z unitary')
        #print(mcalc.uzs[1])
        
        #====== U_QCC already includes the entanglers. This code is not required
        a = tq.Variable("tau_0") # Why do we pick tau ??? This is for H2. What about others?
        
        Ux = []
        #multiple_circuits = False
        #if ( multiple_circuits ):
        #    for e in self.comp_entanglers:
        #
        #        U = construct_QMF_ansatz(4) # Why is 4 selected?
        #
        #        U += tq.gates.ExpPauli(paulistring=tq.PauliString.from_string(self.formatEntangler(e)), angle=a)
        #        Ux.append(U)
        #        
        #else:
        #    U = construct_QMF_ansatz(4) # Why is 4 selected?
        #    for e in self.comp_entanglers:
        #
        #        U += tq.gates.ExpPauli(paulistring=tq.PauliString.from_string(self.formatEntangler(e)), angle=a)
        #        
            # Single circuit
        #    Ux.append(U)
        
        Ux.append(self.comp_U_QCC)
        
        print("============= {} Circuits to execute =============".format(len(Ux)))
        
        c_i = 0
        for circuit in Ux:
            print("Circuit #", c_i)
            c_i = c_i + 1
              
            print(circuit)
            E = tq.ExpectationValue(H=H, U=circuit)
            vars =self.comp_initial_vals
            print(tq.simulate(E, variables=vars))
            
            #print("Calculated vars here      : ", vars)
        
    # For each selected entanglers we will need to compute a separate circuit
    # We we need to create an array of gates
    # we will need to transform the result from Step 3 into a result that can be used here
    #  [X0 Y1 X2 X3] ==> X(0)Y(1)X(2)X(3)

    def formatEntangler(self,e):
        toString = ""
        for t in e.terms:
            for c,axis in t:
                toString += axis + "(" + str(c) + ")"
        return(toString)
            
            

In [16]:
obj = p2t5()

#obj.prepareEntanglers("h2", geometry=2.5, n_ents = 1)
obj.prepareEntanglers("h2", geometry=2.5, n_ents = 1, qubit_transf = "jw")
obj.calculateCircuit()

#obj.prepareEntanglers("h2o", geometry=1,  n_ents = 6, basis="6-31g")
#obj.prepareEntanglers("h2o", geometry=1,  n_ents = 6, active_orbitals = {'B1':[0,1], 'A1':[2,3]})
#obj.prepareEntanglers("h2o", geometry=1,  n_ents = 6, basis="6-31g", active_orbitals = {'B1':[0,1], 'A1':[2,3]})
obj.prepareEntanglers("h2o", geometry=1,  n_ents = 6, basis="6-31g", active_orbitals = {'B1':[0,1], 'A1':[2,3]}, qubit_transf = "jw")
obj.calculateCircuit()

#obj.prepareEntanglers("lih")
#obj.prepareEntanglers("h4")
#obj.prepareEntanglers("n2")
#obj.prepareEntanglers("nh3")

['H2', 'H2O', 'LIH', 'H4', 'N2', 'NH3']
Mapping selected: jw
1) Initialise the molecule object in a given basis and with a provided mapping
	4 orbitals
2) Optionally apply tapering
	1 orbitals
3) Get commuting groups
	2 commuting groups, a 1.50 x reduction from 3 terms
4) For each commuting group get the unitary for transforming to qwc



5) Apply unitaries to get the qwcs



6) Get unitaries for rotating everything to the measurable z-basis



Obtained geometry for h2 is 0.7


Calculating QCC for  H2
Basis: sto-3g
Geometry: 0.7
Entanglers: 1
Active Orbitals: None

Hamiltonian has 15 terms

Full QCC Circuit from Step 3: 
 circuit: 
Rx(target=(0,), parameter=beta_0)
Rz(target=(0,), parameter=gamma_0)
Rx(target=(1,), parameter=beta_1)
Rz(target=(1,), parameter=gamma_1)
Rx(target=(2,), parameter=beta_2)
Rz(target=(2,), parameter=gamma_2)
Rx(target=(3,), parameter=beta_3)
Rz(target=(3,), parameter=gamma_3)
Exp-Pauli(target=(0, 1, 2, 3), control=(), parameter=tau_0, paulistring=X(0)Y(1)X(2)X(3))

Initial Values: {beta_0: 3.141592653589793, gamma_0: 0.0, beta_1: 3.141592653589793, gamma_1: 0.0, beta_2: 0.0, gamma_2: 0.0, beta_3: 0.0, gamma_3: 0.0, tau_0: 0.0}
Optimizer: <class 'tequila.optimizers.optimizer_scipy.OptimizerSciPy'> 
backend         : qulacs
samples         : None
save_history    : True
noise           : None

Method          : BFGS
Objective       : 1 expectationvalues
gradient        : 18 expectationvalues

active v


Obtained geometry for h2o is 1


Calculating QCC for  H2O
Basis: 6-31g
Geometry: 1
Entanglers: 6
Active Orbitals: {'B1': [0, 1], 'A1': [2, 3]}
There are known issues with some psi4 methods and frozen virtual orbitals. Proceed with fingers crossed for hf.
There are known issues with some psi4 methods and frozen virtual orbitals. Proceed with fingers crossed for fci.

Hamiltonian has 185 terms

Full QCC Circuit from Step 3: 
 circuit: 
Rx(target=(0,), parameter=beta_0)
Rz(target=(0,), parameter=gamma_0)
Rx(target=(1,), parameter=beta_1)
Rz(target=(1,), parameter=gamma_1)
Rx(target=(2,), parameter=beta_2)
Rz(target=(2,), parameter=gamma_2)
Rx(target=(3,), parameter=beta_3)
Rz(target=(3,), parameter=gamma_3)
Rx(target=(4,), parameter=beta_4)
Rz(target=(4,), parameter=gamma_4)
Rx(target=(5,), parameter=beta_5)
Rz(target=(5,), parameter=gamma_5)
Rx(target=(6,), parameter=beta_6)
Rz(target=(6,), parameter=gamma_6)
Rx(target=(7,), parameter=beta_7)
Rz(target=(7,), parameter=gamma_7)
Exp-Pauli

AttributeError: 'MeasurementGrouper' object has no attribute 'uqwcs'